## Train a model with bike rental data using XGBoost algorithm
### Training log1p(count) dataset
###  Model is trained with XGBoost installed in notebook instance
###  In the later examples, we will train using SageMaker's XGBoost algorithm

In [ ]:
# Install xgboost in notebook instance.
#### Command to install xgboost
# !pip install xgboost==0.90

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# XGBoost 
import xgboost as xgb

import matplotlib.pyplot as plt

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

<h2>Kaggle Bike Sharing Demand Dataset</h2>

Modified 'count' to log1p(count) for training

Log can be used when target represents a count (that is non-negative values)

Model now predicts as log1p(count). We need to convert it back to actual count using expm1(predicted_target)

Reference:
https://www.kaggle.com/apapiu/predicting-bike-sharing-with-xgboost by Alexandru Papiu

To download dataset, sign-in and download from this link:
https://www.kaggle.com/c/bike-sharing-demand/data <br>


Input Features: ['season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek','hour']<br>
Target Feature: [<b>log1p('count')</b>]<br>
Objective: <quote>You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period (Ref: Kaggle.com)</quote>

In [ ]:
column_list_file = '../Data/bike_train_column_list.txt'
train_file = '../Data/bike_train.csv'
validation_file = '../Data/bike_validation.csv'
test_file = '../Data/bike_test.csv'

In [ ]:
columns = ''
with open(column_list_file,'r') as f:
    columns = f.read().split(',')

In [ ]:
columns

In [ ]:
# Specify the column names as the file does not have column header
df_train = pd.read_csv(train_file,names=columns)
df_validation = pd.read_csv(validation_file,names=columns)

In [ ]:
df_train.head()

In [ ]:
df_validation.head()

In [ ]:
X_train = df_train.iloc[:,1:] # Features: 1st column onwards 
y_train = df_train.iloc[:,0].ravel() # Target: 0th column

X_validation = df_validation.iloc[:,1:]
y_validation = df_validation.iloc[:,0].ravel()

In [ ]:
# XGBoost Training Parameter Reference: 
#   https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
#regressor = xgb.XGBRegressor(max_depth=5,eta=0.1,subsample=0.7,num_round=150)
regressor = xgb.XGBRegressor(max_depth=5,n_estimators=150)

In [ ]:
regressor

In [ ]:
regressor.fit(X_train,y_train, eval_set = [(X_train, y_train), (X_validation, y_validation)])

In [ ]:
df_train['count'].describe()

In [ ]:
eval_result = regressor.evals_result()

In [ ]:
training_rounds = range(len(eval_result['validation_0']['rmse']))

In [ ]:
plt.scatter(x=training_rounds,y=eval_result['validation_0']['rmse'],label='Training Error')
plt.scatter(x=training_rounds,y=eval_result['validation_1']['rmse'],label='Validation Error')
plt.grid(True)
plt.xlabel('Iteration')
plt.ylabel('RMSE')
plt.title('Training Vs Validation Error')
plt.legend()
plt.show()

In [ ]:
xgb.plot_importance(regressor)
plt.show()

In [ ]:
# Updated - Changed to validation dataset
# Compare actual vs predicted performance with dataset not seen by the model before
df = pd.read_csv(validation_file,names=columns)

In [ ]:
df.head()

In [ ]:
X_test = df.iloc[:,1:]
print(X_test[:5])

In [ ]:
result = regressor.predict(X_test)

In [ ]:
result[:5]

In [ ]:
df.head()

In [ ]:
df['count_predicted'] = result

In [ ]:
df.head()

In [ ]:
# Negative Values are predicted
df['count_predicted'].describe()

In [ ]:
df[df['count_predicted'] < 0]

In [ ]:
def adjust_count(x):
    if x < 0:
        return 0
    else:
        return x

In [ ]:
df['count_predicted'] = df['count_predicted'].map(adjust_count)

In [ ]:
df[df['count_predicted'] < 0]

In [ ]:
df['count'] = df['count'].map(np.expm1)
df['count_predicted'] = df['count_predicted'].map(np.expm1)

In [ ]:
# Actual Vs Predicted
plt.plot(df['count'], label='Actual')
plt.plot(df['count_predicted'],label='Predicted')
plt.xlabel('Sample')
plt.ylabel('Count')
plt.xlim([100,150])
plt.title('Validation Dataset - Predicted Vs. Actual')
plt.legend()
plt.show()

In [ ]:
# Over prediction and Under Prediction needs to be balanced
# Training Data Residuals
residuals = (df['count'] - df['count_predicted'])

plt.hist(residuals)
plt.grid(True)
plt.xlabel('Actual - Predicted')
plt.ylabel('Count')
plt.title('Residuals Distribution')
plt.axvline(color='r')
plt.show()

In [ ]:
value_counts = (residuals > 0).value_counts(sort=False)
print(' Under Estimation: {0:.2f}'.format(value_counts[True]/len(residuals)))
print(' Over  Estimation: {0:.2f}'.format(value_counts[False]/len(residuals)))

In [ ]:
import sklearn.metrics as metrics
print("RMSE: {0:.2f}".format(metrics.mean_squared_error(df['count'],
                                                    df['count_predicted'])**.5))

In [ ]:
# Metric Use By Kaggle
def compute_rmsle(y_true, y_pred):
    if type(y_true) != np.ndarray:
        y_true = np.array(y_true)
        
    if type(y_pred) != np.ndarray:
        y_pred = np.array(y_pred)
     
    return(np.average((np.log1p(y_pred) - np.log1p(y_true))**2)**.5)

In [ ]:
print("RMSLE: {0:.2f}".format(compute_rmsle(df['count'],df['count_predicted'])))

In [ ]:
# Prepare Data for Submission to Kaggle
df_test = pd.read_csv(test_file,parse_dates=['datetime'])

In [ ]:
df_test.head()

In [ ]:
X_test =  df_test.iloc[:,1:] # Exclude datetime for prediction

In [ ]:
X_test.head()

In [ ]:
result = regressor.predict(X_test)

In [ ]:
result[:5]

In [ ]:
np.expm1(result)

In [ ]:
# Convert result to actual count
df_test["count"] = np.expm1(result)

In [ ]:
df_test.head()

In [ ]:
df_test[df_test["count"] < 0]

In [ ]:
df_test[['datetime','count']].to_csv('../Data/predicted_count.csv',index=False)